
requirment:
python==3.6.5
flask

In [1]:
import importlib
import featurebinning as fb
importlib.reload(fb)

import pandas as pd

In [2]:
data1 = pd.read_pickle("./data_sample.pkl")
display(data1.head(2))

,var0,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var40,var41,var42,var43,var44,var45,var46,var47,var48,target
188489,1.5,1.5,600.0,3.0,1.0,2.2,2.666667,4.0,6.0,200.0,...,2.3061,0.0,2.000000,0.0,5.5,500.0,3.6,2.5,6000.0,0
133130,0.5,1.5,200.0,NaN,1.0,NaN,50.000000,0.0,16.0,-99998.0,...,69.1830,-10000000.0,1.111111,0.0,22.5,100.0,NaN,0.5,8000.0,0


In [3]:
wt = fb.WideTable(data1)
print(wt.label_mapping)
wt.label_mapping.update({"var0":'变量0',"var2":'变量2'})
print(wt.label_mapping)

{}
{'var0': '变量0', 'var2': '变量2'}


In [4]:
wt.run_app()

server running on 127.0.0.1:50000 * Serving Flask app "binning_server" (lazy loading)



<Thread(Thread-57, started daemon 3328)>

 * Environment: development
 * Debug mode: off


In [3]:
%%HTML
<iframe src="http://127.0.0.1:50000/test?var=var7" width="90%" height="720px" style="background-color:antiquewhite"></iframe>

<Logger werkzeug (INFO)>

In [5]:
df_raw = pd.read_pickle("./data_sample.pkl")
var_bin,bin_label,var_woe,woe_bin_mapping = wt.var_cut(df_raw,"var0",calculate_woe=True)
display(var_bin.head())
display(bin_label)
display(var_woe.head())
display(woe_bin_mapping)
var_bin,bin_label,var_woe,woe_bin_mapping

var0: woes set to zero for very small bins
   total  bad  good
2     10    1     9


188489    1
133130    0
118749    0
4345      0
132270    0
dtype: int8

{0: '(-99998.00200000001, 0.5]', 1: '(0.5, 20.501]', 2: '(20.501, inf]'}

188489   -0.112290
133130   -0.031534
118749   -0.031534
4345     -0.031534
132270   -0.031534
dtype: float64

{-1: 0.1428944938022283,
 0: -0.03153395573963996,
 1: -0.11229036298730305,
 2: 0.0}

In [9]:
var_bin,bin_label = wt.var_cut(df_raw,"var0",calculate_woe=False)
display(var_bin.head())
display(bin_label)

188489    1
133130    0
118749    0
4345      0
132270    0
dtype: int8

{0: '(-99998.00200000001, 0.5]', 1: '(0.5, 20.501]', 2: '(20.501, inf]'}

In [ ]:
# 部署
bin_info = pd.read_csv("./bin_info.csv",index_col=0)
bins=bin_info.loc[[f for f in feature_list_train if f not in categorical_encode]].dropna(how="all",axis=1)
model_data = {"bins":bins.to_json(),"model":model_lgb.model_to_string()}
joblib.dump(model_data,"./a4_app_md1027_150631.pkl")

In [ ]:
import json
import pandas as pd
import joblib
import os
import numpy as np
from cachetools import LRUCache
from cachetools import cached
import lightgbm as lgb

# 缓存
lrucache = LRUCache(maxsize=50)
@cached(lrucache)
def get_lgbm(fname):
    print(fname)
    return joblib.load(fname)

cpath = "./" #os.path.dirname(__file__)
filename = 'a4_app_md1027_150631.pkl'
full_name = os.path.join(cpath, filename)
model_data = get_lgbm(full_name)

LGBM = lgb.Booster(train_set=lgb.Dataset(np.array([[0]])))
LGBM.model_from_string(model_data["model"])
BIN_INFO = pd.DataFrame.from_dict(json.loads(model_data["bins"]))


def encode(v,bins):
    if pd.isnull(v):
        return -1
    for i,edge in enumerate(bins[1:-1]):
        if v <= edge:
            return i
    return len(bins)-2


FEAURE_IN = "var0,var1,var2".split(",")

def model_predict(**kwargs):
    for f in FEAURE_IN:
        if f not in kwargs:
            raise Exception(f"feature {f} not in input")
    for f in kwargs:
        if f in bin_info.columns:
            kwargs[f] = encode(kwargs[f],BIN_INFO[f])

    